# Simple data visualization: the bar chart

In this notebook we'll be comparing as many Python visualization libraries as possible with one simple metric: how easy is it to make a bar chart? We'll also look a bit into customization with each library, and explore the pain points a bit. 

_Note: if you're viewing this on GitHub, charting libraries that output Javascript (Bokeh, Holoviews, Altair) won't display. Try viewing the notebook on [my nbviewer page](http://nbviewer.jupyter.org/github/dovinmu/python-viz-notebooks/tree/master/) instead._

First, let's load a pandas DataFrame of the data we'll be using and clean it a bit. There's a CSV in the repository that lists over 4,000 cities and towns in the United States. For now, let's just take the cities that are over a certain size and plop that into various charting libraries. 

In [ ]:
import pandas
%matplotlib inline

df = pandas.read_csv('data/US_cities.csv', index_col="AccentCity")
# select cities with a population greater than 750,000 and convert units to millions
large_cities = df[df["Population"] > 750000][["Population"]] / 1000000
# sort the list by population
large_cities.sort_values("Population", inplace=True)
# we don't care about all the other data, so discard everything but the population
large_cities = pandas.Series(large_cities["Population"])
# also don't care about the name of the cities column in the CSV
del large_cities.index.name

large_cities

You could debate the decision to change the population values to floats that count millions of people, versus integers that store the exact population number. For our purposes right now, though, we don't need the exact number, and it will remove the headache of tweaking the Y-axis so it's more readable. 

### Matplotlib
Matplotlib is both amazingly versatile, and also generally more verbose than the other packages below (which usually will use it as a backend rendering target for static graphs). You need to manually specify a few things that other packages just handle.

In [ ]:
import matplotlib.pyplot as plt

# manually tell matplotlib where the bars will go, so we'll space them out every 10 pixels
left = [i * 10 for i in range(len(large_cities))]
plt.bar(left, large_cities.values, width=5, tick_label=large_cities.index)
# make the city names vertical so we can read them
plt.xticks(rotation="vertical")
plt.ylabel("Population (m)")

And we get this decent-looking bar chart.

### Pandas + Matplotlib

Pandas is what we loaded the data in with, and it has plotting functions that use matplotlib as the backend but are aware of how your dataframe is structured. This is all we need to do to make an almost identical chart as the one above.

In [ ]:
large_cities.plot.bar()
plt.ylabel("Population (m)")

Because pandas is plotting to matplotlib on the backend, you can still tweak it to your heart's content with commands directly to matplotlib like in the second line above. 

### Seaborn

In [ ]:
import seaborn
seaborn.barplot(x=large_cities.index, y=large_cities)
plt.xticks(rotation="vertical")
plt.ylabel("Population (m)")

### Bokeh

Here's how to make a bar chart similar to the ones above in Bokeh. It's actually not trivial.

In [ ]:
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

output_notebook()

cities = list(large_cities.index)
populations = list(large_cities.values)

source = ColumnDataSource(data=dict(cities=cities, populations=populations))

p = figure(x_range=cities,  toolbar_location=None)
p.vbar(x='cities', top='populations', width=0.9, source=source, 
       line_color='white')
p.xaxis.major_label_orientation = 3.1415962 / 2
show(p)

A few pain points:
 * Have to manually convert pandas DataFrame into a Bokeh-compatible data source
 * Even using bokeh's native data type, have to specify several arguments about how to chart it. This complaint mostly comes down to the fact that vbar is not supposed to be a high-level charting function, which we'll get to in a second.
 * Default vertical setting for x-axis labels reads as backwards, so we have to manually specify the rotation in radians to get it to look right
 
Generally, the code is just pretty verbose, since Bokeh deprecated its high-level charting functions. These are now being provided by Holoviews, which is the next library we'll try out.

### References

Bokeh bar chart: http://bokeh.pydata.org/en/latest/docs/gallery/bar_colormapped.html

